In [1]:
import tkinter as tk
import tkinter.messagebox

In [2]:
# 用于记录游戏双方
class Game:
    HUMAN = -1 # 人类方
    COMP = 1 # 电脑方

In [3]:
import copy
# 实现 minimax 算法
class Minimax:
    def __init__(self, state):
        '''
        state 用于存储当前棋盘状态
        '''
        self.state = state
    def get_win(self, state, player):
        '''
        return Game.HUMAN 表示人类赢
        return Game.COMP 表示电脑赢
        return 0 表示平局
        '''
        wins = [
            [state[0][0], state[0][1], state[0][2]],
            [state[1][0], state[1][1], state[1][2]],
            [state[2][0], state[2][1], state[2][2]],
            [state[0][0], state[1][0], state[2][0]],
            [state[0][1], state[1][1], state[2][1]],
            [state[0][2], state[1][2], state[2][2]],
            [state[0][0], state[1][1], state[2][2]],
            [state[0][2], state[1][1], state[2][0]]
        ]
        if [player, player, player] in wins:
            return True
        return False
    
    def get_all_empty_cell(self, state):
        '''
        用于提取出没有被访问的节点
        '''
        cells = []
        for i in range(len(state)):
            for j in range(len(state[i])):
                if state[i][j] == 0:
                    cells.append([i, j])
        return cells
    def run(self, depth, play):
        '''
        minimax 具体实现
        '''
        def minimax(state, depth, player):
            if player == Game.COMP:
                best = [-1, -1, -10000]
            else: 
                best = [-1, -1, 10000]
            if depth == 0 or self.get_win(state, player) or self.get_win(state, -player):
                if self.get_win(state, Game.HUMAN):
                    return [-1, -1, -1]
                elif self.get_win(state, Game.COMP):
                    return [-1, -1, 1]
                return [-1, -1, 0]
            for cell in self.get_all_empty_cell(state):
                x, y = cell[0], cell[1]
                state[x][y] = player
                score = minimax(state, depth-1, -player)
                state[x][y] = 0
                score[0], score[1] = x, y
                if player == Game.COMP:
                    if score[2] > best[2]:
                        best = score
                else:
                    if score[2] < best[2]:
                        best = score
            return best
        return minimax(copy.deepcopy(self.state), depth, play)

In [4]:
  class Checkerboard:
    def __init__(self):
        '''
        画图
        '''
        self.circle = True
        # 当为False时才能下棋 当为True时阻止继续下棋
        self.win = False
        # 用于计数 当为9时表示已经下满所有的格子
        self.num = 0
        self.window = tk.Tk()
        self.window.title = '井子棋'
        self.canvas = tk.Canvas(self.window, bg='green', height=200, width=200)
        # 画出井字棋
        for i in range(4):
            self.canvas.create_line(10 + i * 60, 10, 10 + i * 60, 190)
            self.canvas.create_line(10, 10 + i * 60, 190, 10 + i * 60)
        self.canvas.pack()
        self.canvas.bind('<Button-1>', self.human_go)
        # 用来记录井字棋里面的数据
        self.data = [
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0]
        ]
        # 加载minimax算法
        self.minimax = Minimax(self.data)
    def human_go(self, event):
        '''
        人类选手下棋逻辑
        '''
        # 下满或者出现赢方停止下棋
        if self.win or self.num > 9:
            return
        if event:
            # 拿到点击位置
            x = event.x
            y = event.y
            # 找到点击位置对应的坐标
            if x > 10 and y > 10 and x < 190 and y < 190:
                x = (x - 10) // 60
                y = (y - 10) // 60
            else:
                return
            if self.data[y][x] != 0:
                return
            # 下棋
            self.drawChess(x, y)
            # 判断是否赢了
            if self.minimax.get_win(self.data, Game.HUMAN):
                tkinter.messagebox.showinfo(title = '胜利', message = '你赢了')
                self.win = True
                return
            # 交给ai走棋
            self.ai_turn()

    def ai_turn(self):
        '''
        ai 走棋逻辑
        '''
        # 下满或者出现赢方停止下棋
        if self.num > 9 or self.win:
            return
        # 利用minimax算法求出合适的坐标
        best = self.minimax.run(9 - self.num, Game.COMP)
        # 下棋
        self.drawChess(y=best[0], x=best[1], game=Game.COMP)
        # 判断是否赢了
        if self.minimax.get_win(self.data, Game.COMP):
            tkinter.messagebox.showinfo(title = '胜利', message = '电脑胜利')
            self.win = True
        
    def drawChess(self, x=None, y=None, game=Game.HUMAN):
        '''
        下棋逻辑 其中 x 为横坐标，y 为纵坐标
        ---------------- x
        |
        |
        |
        |
        |
        y
        '''
        # 判断当前应该下什么棋子，两种棋子交替下
        if self.circle :
            self.canvas.create_oval(20 + x * 60, 20 + y * 60, (x + 1) * 60, (y + 1) * 60, width = 5)
            self.data[y][x] = game
        else:
            self.canvas.create_line(20 + x * 60, 20 + y * 60, (x + 1) * 60, (y + 1) * 60, width = 5)
            self.canvas.create_line((x + 1) * 60, 20 + y * 60, 20 + x * 60, (y + 1) * 60, width = 5)
            self.data[y][x] = game
        # 改变棋子
        self.circle = not self.circle
        # 棋子数量加 1
        self.num += 1
    def show(self):
        self.window.mainloop()

In [ ]:
Checkerboard().show()